
## Subconsultas en SQL

Una **subconsulta** es una consulta dentro de otra consulta SQL que se utiliza para realizar consultas más complejas. Estas subconsultas se anidan dentro de las instrucciones `SELECT`, `INSERT`, `UPDATE` o `DELETE` y pueden ayudar a filtrar resultados, calcular valores, y seleccionar datos según condiciones más específicas.

### Características de las Subconsultas
1. **Ubicación**: Pueden estar en la cláusula `SELECT`, `WHERE`, `FROM` o `HAVING` de una consulta.
2. **Tipos**: Las subconsultas pueden ser:
   - **Escalares**: Devuelven un solo valor.
   - **De una sola columna**: Devuelven una lista de valores en una columna específica.
   - **De varias columnas**: Devuelven una o más filas y columnas.
3. **Operadores Relacionales**: Los operadores `IN`, `ANY`, `ALL`y `EXISTS` son herramientas útiles en SQL para comparar los resultados de subconsultas con los resultados de una consulta principal. Aquí te explico cada uno:

   1. ***IN***
   El operador IN se utiliza para verificar si un valor se encuentra dentro de un conjunto de valores devueltos por una subconsulta. Es una forma de simplificar las condiciones WHERE.

   2. **ANY**
   El operador ANY compara un valor con un conjunto de valores devueltos por una subconsulta. Si al menos una de las comparaciones es verdadera, la condición se cumple.

   3. **ALL**
   El operador ALL compara un valor con todos los valores devueltos por una subconsulta. La condición se cumple solo si la comparación es verdadera para todos los elementos del conjunto.

   4. **EXISTS**
   El operador EXISTS verifica si hay al menos una fila devuelta por una subconsulta. Se utiliza comúnmente en combinación con una subconsulta correlacionada.

<h2>1. Subconsulta en la cláusula SELECT</h2>

<p>Generamos las tablas y los datos que vamos a usar como ejemplo:</p>

In [ ]:
%load_ext sql

# Conectarse a una base de datos SQLite en memoria
%sql sqlite:///:memory:

In [ ]:
%%sql
CREATE TABLE empleados (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    salario REAL NOT NULL,
    departamento_id INTEGER
);

In [ ]:
%%sql
CREATE TABLE departamentos (
    id INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL
);

In [14]:
%%sql
INSERT INTO empleados (nombre, salario, departamento_id) 
VALUES ('Ana', 3000, 1),
    ('Luis', 4500, 2),
    ('Maria', 5000, 2),
    ('Carlos', 3500, 3),
    ('Sara', 4000, Null);

 * sqlite:///:memory:
5 rows affected.


[]

In [5]:
%%sql
INSERT INTO departamentos (id, nombre) VALUES 
    (1, 'Recursos Humanos'),
    (2, 'Ventas'),
    (3, 'Tecnología');

 * sqlite:///:memory:
3 rows affected.


[]

<p>Queremos mostrar a cada empleado con su salario y el nombre de su departamento. <p>Usaremos una subconsulta para obtener el nombre del departamento.</p>

In [15]:
%%sql
SELECT * FROM empleados;


 * sqlite:///:memory:
Done.


id,nombre,salario,departamento_id
1,Ana,3000.0,1
2,Luis,4500.0,2
3,Maria,5000.0,2
4,Carlos,3500.0,3
5,Sara,4000.0,None


In [9]:
%%sql
SELECT * FROM departamentos;

 * sqlite:///:memory:
Done.


id,nombre
1,Recursos Humanos
2,Ventas
3,Tecnología


In [16]:
%%sql
SELECT 
    e.nombre AS empleado,
    e.salario AS salario,
    (SELECT nombre 
     FROM departamentos d 
     WHERE d.id = e.departamento_id) AS departamento
FROM empleados e;

 * sqlite:///:memory:
Done.


empleado,salario,departamento
Ana,3000.0,Recursos Humanos
Luis,4500.0,Ventas
Maria,5000.0,Ventas
Carlos,3500.0,Tecnología
Sara,4000.0,None


<h2>2. Subconsulta en la cláusula WHERE</h2>
<p>Queremos listar a los empleados que ganan más que el salario promedio.</p>

In [17]:
%%sql
SELECT 
    nombre,
    salario
FROM empleados
WHERE salario > (SELECT AVG(salario) FROM empleados);

 * sqlite:///:memory:
Done.


nombre,salario
Luis,4500.0
Maria,5000.0


<h2>3. Subconsulta en una cláusula UPDATE</h2>
<p>Vamos a incrementar el salario de los empleados que ganan menos que el salario promedio.</p>

In [ ]:
%%sql
SELECT * FROM empleados;

In [18]:
%%sql
UPDATE empleados
SET salario = salario * 1.1
WHERE salario < (SELECT AVG(salario) FROM empleados);

 * sqlite:///:memory:
2 rows affected.


[]

<h2>4. Subconsulta en la cláusula FROM</h2>
<p>Calcular la cantidad de empleados por departamento</p>

In [19]:
%%sql
SELECT 
    departamento_id,
    cantidad_empleados
FROM 
    (SELECT departamento_id, COUNT(*) AS cantidad_empleados
     FROM empleados
     GROUP BY departamento_id) subconsulta;

 * sqlite:///:memory:
Done.


departamento_id,cantidad_empleados
None,1
1,1
2,2
3,1


<h2>5. Subconsulta en una sentencia INSERT</h2>
<p>Vamos a insertar un nuevo empleado en el departamento con el menor número de empleados.</p>

In [20]:
%%sql
SELECT * FROM empleados;

 * sqlite:///:memory:
Done.


id,nombre,salario,departamento_id
1,Ana,3300.0000000000005,1
2,Luis,4500.0,2
3,Maria,5000.0,2
4,Carlos,3850.0000000000005,3
5,Sara,4000.0,None


In [ ]:
%%sql
INSERT INTO empleados (nombre, salario, departamento_id)
VALUES ('Jorge', 2800, 
        (SELECT departamento_id 
         FROM (SELECT departamento_id, COUNT(*) AS empleados
               FROM empleados
               GROUP BY departamento_id
               ORDER BY empleados ASC LIMIT 1)
        )
);


### Ejercicios de Subconsultas en la clausula WHERE

A continuación se muestran 10 ejercicios de subconsultas en SQL para practicar:


In [ ]:
%%sql
CREATE TABLE Clientes (
    cliente_id INTEGER PRIMARY KEY,
    nombre VARCHAR(50),
    apellido VARCHAR(50),
    pais VARCHAR(50),
    edad INTEGER
);

CREATE TABLE Pedidos (
    pedido_id INT PRIMARY KEY,
    producto VARCHAR(50),
    cantidad INT,
    cliente_id INT,
    pendiente BOOLEAN,
    FOREIGN KEY (cliente_id) REFERENCES Clientes(cliente_id)
);

INSERT INTO Clientes (cliente_id, nombre, apellido, pais, edad) VALUES
(1, 'Juan', 'Pérez', 'España', 30),
(2, 'Ana', 'García', 'México', 25),
(3, 'Luis', 'Martínez', 'Argentina', 35),
(4, NULL, NULL,NULL, NULL),
(5, 'Carlos', 'González', 'Colombia', 40),
(6, 'Laura', 'Fernández', 'Chile', 22),
(7, 'Pedro', 'Sánchez', 'España', 31),
(8, 'Jhon', 'Doe', 'Reino Unido', 29),
(9, 'Isabel', 'Núñez', 'México', 26),
(10, 'Miguel', 'Romero', 'Perú', 33);

INSERT INTO Pedidos (pedido_id, producto, cantidad, cliente_id, pendiente) VALUES
(1, 'Teclado', 5, 1, FALSE),
(2, 'Ratón', 3, 2, TRUE),
(3, 'Teclado', 4, 3, FALSE),
(4, 'Ratón', 2, NULL, TRUE),
(5, 'Monitor', 1, 5, FALSE),
(6, 'Teclado', 2, 6, TRUE),
(7, 'Ratón', 4, 7, FALSE),
(8, 'Teclado', 3, 8, TRUE),
(9, 'Monitor', 2, 9, FALSE),
(10, 'Ratón', 5, 10, TRUE);


1. **Obtener el nombre y apellido de los clientes que tienen algún pedido pendiente:**

In [ ]:
%%sql


2. **Listar los productos pedidos por clientes de España:**:

In [ ]:
%%sql



3. **Clientes que han hecho pedidos de más de 3 productos en un solo pedido:**

In [ ]:
%%sql



4. **Clientes cuyo nombre, apellido o país son desconocidos (valores NULL):**

In [ ]:
%%sql


5. **Edad promedio de los clientes con pedidos pendientes:**

In [ ]:
%%sql


6. **Obtener el país de los clientes que han pedido un 'Monitor':**

In [ ]:
%%sql


7. **Clientes que han hecho pedidos tanto de 'Teclado' como de 'Ratón':**

In [ ]:
%%sql


8. **Clientes con edad superior al promedio de los clientes que viven en México:**

In [ ]:
%%sql


9. **Listar los clientes que no tienen pedidos registrados:**

In [ ]:
%%sql


10. **Obtener el cliente que tiene el pedido con la mayor cantidad de productos:**

In [ ]:
%%sql
